In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# hyperparameters
num_epochs = 10
batch_size = 100
learning_rate = 0.001

# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
)

# cifar10 data
train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                           train=True,
                                           transform=transform,
                                           download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                           train=False,
                                          transform=transform,
                                           download=True)

# dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                         shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                         shuffle=False)


examples = iter(test_loader)
example_data, example_targets = next(examples)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(64*4*4, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        # N, 3, 32, 32
        x = F.relu(self.conv1(x)) # N, 32, 30, 30
        x = self.pool(x) # N, 32, 15, 15
        x = F.relu(self.conv2(x)) # N, 64, 13, 13
        x = self.pool(x) # N, 64, 6, 6
        x = F.relu(self.conv3(x)) # N, 64, 4, 4
        x = torch.flatten(x, 1) # 64 * 4 *4 = 1024
        x = F.relu(self.fc1(x)) # N, 64
        x = self.fc2(x)
        return x

model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'[{epoch+1}] loss: {running_loss / n_total_steps: .3f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

[1] loss:  1.616
[2] loss:  1.253
[3] loss:  1.072
[4] loss:  0.954
[5] loss:  0.872
[6] loss:  0.805
[7] loss:  0.752
[8] loss:  0.704
[9] loss:  0.667
[10] loss:  0.628
Finished Training


In [14]:
len(train_dataset)

50000

In [15]:
len(train_loader)

500

In [16]:
loaded_model = ConvNet()
loaded_model.load_state_dict(torch.load(PATH))
loaded_model.to(device)
loaded_model.eval()

with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = loaded_model(images)

    _, preds = torch.max(outputs, 1)
    n_correct += (preds == labels).sum().item()
    n_samples += labels.shape[0]

  acc = 100* (n_correct / n_samples)
  print(f'accuracy: {acc}%')

accuracy: 73.21%
